# MPRA Find Significant Activity on All Versions and Pools

In [1]:
library(MPRAnalyze)
library(tidyverse)
library(ggplot2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Pool 1

In [2]:
# load MPRAObjects
pool_alpha <- read.table("../results/pool1_mpranalyze_results.txt")

In [3]:
# load DNA CPM counts
pool_cpm <- read.table("../results/pool1_dna_counts_cpm.txt")

In [4]:
# load results from allelic significance
pool_allelic <- read.table("../results/pool1_mpranalyze_allelic_results_merged.txt", header=TRUE)

In [5]:
head(pool_alpha)

,snp,type,alpha
,<chr>,<chr>,<dbl>
1,rs11583631,ref,1.425401
2,rs11583631,alt,1.328563
3,rs11584419,ref,1.476491
4,rs11584419,alt,1.412684
5,rs749435,ref,2.005698
6,rs749435,alt,2.140291


In [6]:
head(pool_cpm)

,chrom,hg19_pos,snp,allele,mutation,testcre,V1_DNA_ref_count,V1_DNA_alt_count,V2_DNA_ref_count,V2_DNA_alt_count,V1_DNA_ref_CPM,V1_DNA_alt_CPM,V2_DNA_ref_CPM,V2_DNA_alt_CPM,DNA_CPM_QC
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
1,chr1,6147298,rs11583631,C,T,chr1_6147297_rs11583631_C_T,39,36,193,234,5.812979,5.365827,6.337149,7.683383,FALSE
2,chr1,6147341,rs11584419,A,C,chr1_6147340_rs11584419_A_C,29,43,191,368,4.322472,6.409182,6.271479,12.083269,FALSE
3,chr1,6157297,rs749435,T,C,chr1_6157296_rs749435_T_C,870,899,5367,4969,129.674157,133.996629,176.225288,163.156969,TRUE
4,chr1,6258794,rs3789524,A,G,chr1_6258793_rs3789524_A_G,9743,14828,46568,70300,1452.201509,2210.124600,1529.058916,2308.298441,TRUE
5,chr1,6259025,rs3789522,C,T,chr1_6259024_rs3789522_C_T,7689,6910,35203,29103,1146.051258,1029.940720,1155.889474,955.596153,TRUE
6,chr1,6260663,rs11121478,A,C,chr1_6260662_rs11121478_A_C,128,43,952,499,19.078497,6.409182,31.258892,16.384650,TRUE


In [7]:
head(pool_allelic)

,snp,logFC,lrt_pval,lrt_nlog10_pval
,<chr>,<dbl>,<dbl>,<dbl>
1,rs11583631,-0.04727376,6.748780e-01,0.1707748
2,rs11584419,0.04456964,6.565204e-01,0.1827518
3,rs749435,-0.06529871,8.405816e-02,1.0754201
4,rs3789524,0.04192126,1.242780e-03,2.9056058
5,rs3789522,-0.16202368,2.199461e-19,18.6576837
6,rs11121478,-0.17450039,3.369730e-02,1.4724049


In [8]:
head(pool_alpha)

,snp,type,alpha
,<chr>,<chr>,<dbl>
1,rs11583631,ref,1.425401
2,rs11583631,alt,1.328563
3,rs11584419,ref,1.476491
4,rs11584419,alt,1.412684
5,rs749435,ref,2.005698
6,rs749435,alt,2.140291


In [9]:
# log-transform alpha values
pool_alpha <- pool_alpha %>% mutate(log2a = log2(alpha))

In [10]:
# calculate z-score with alpha
pool_alpha <- pool_alpha %>% mutate(zscore = (log2a - mean(log2a)) / sd(log2a))

In [11]:
head(pool_alpha)

,snp,type,alpha,log2a,zscore
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,rs11583631,ref,1.425401,0.5113677,-0.098448693
2,rs11583631,alt,1.328563,0.4098670,-0.304637789
3,rs11584419,ref,1.476491,0.5621723,0.004756118
4,rs11584419,alt,1.412684,0.4984384,-0.124713388
5,rs749435,ref,2.005698,1.0041041,0.902498810
6,rs749435,alt,2.140291,1.0978068,1.092847129


In [12]:
# widen table to separate alleletypes and find enhancer significance
pool_alpha_w <- pool_alpha %>% 
    pivot_wider(names_from=type, values_from=c(alpha, log2a, zscore))

In [13]:
# select relevant columns from DNA CPM counts
pool_cpm <- pool_cpm %>% select(snp, DNA_CPM_QC)

In [14]:
head(pool_cpm)

,snp,DNA_CPM_QC
,<chr>,<lgl>
1,rs11583631,FALSE
2,rs11584419,FALSE
3,rs749435,TRUE
4,rs3789524,TRUE
5,rs3789522,TRUE
6,rs11121478,TRUE


In [15]:
# add DNA CPM classifications to v1 pool1 table
pool_alpha_w <- pool_alpha_w %>% left_join(pool_cpm, by = c("snp"))

In [16]:
head(pool_alpha_w)

snp,alpha_ref,alpha_alt,log2a_ref,log2a_alt,zscore_ref,zscore_alt,DNA_CPM_QC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
rs11583631,1.425401,1.328563,0.5113677,0.4098670,-0.098448693,-0.30463779,FALSE
rs11584419,1.476491,1.412684,0.5621723,0.4984384,0.004756118,-0.12471339,FALSE
rs749435,2.005698,2.140291,1.0041041,1.0978068,0.902498810,1.09284713,TRUE
rs3789524,1.422514,1.491660,0.5084428,0.5769184,-0.104390459,0.03471129,TRUE
rs3789522,1.602634,1.360524,0.6804446,0.4441627,0.245015024,-0.23496934,TRUE
rs11121478,1.132672,1.146398,0.1797297,0.1971084,-0.772140080,-0.73683690,TRUE


In [17]:
# classify variants that are enhancers
pool_enhancer <- pool_alpha_w %>% filter(DNA_CPM_QC) %>% mutate(is_enhancer=(zscore_ref > 0 | zscore_alt > 0))

In [18]:
# join enhancer classifications with overall table
pool <- pool_alpha_w %>% left_join(pool_enhancer)

Joining with `by = join_by(snp, alpha_ref, alpha_alt, log2a_ref, log2a_alt,
zscore_ref, zscore_alt, DNA_CPM_QC)`


In [19]:
head(pool)

snp,alpha_ref,alpha_alt,log2a_ref,log2a_alt,zscore_ref,zscore_alt,DNA_CPM_QC,is_enhancer
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>
rs11583631,1.425401,1.328563,0.5113677,0.4098670,-0.098448693,-0.30463779,FALSE,NA
rs11584419,1.476491,1.412684,0.5621723,0.4984384,0.004756118,-0.12471339,FALSE,NA
rs749435,2.005698,2.140291,1.0041041,1.0978068,0.902498810,1.09284713,TRUE,TRUE
rs3789524,1.422514,1.491660,0.5084428,0.5769184,-0.104390459,0.03471129,TRUE,TRUE
rs3789522,1.602634,1.360524,0.6804446,0.4441627,0.245015024,-0.23496934,TRUE,TRUE
rs11121478,1.132672,1.146398,0.1797297,0.1971084,-0.772140080,-0.73683690,TRUE,FALSE


In [20]:
# extract allelic p-values
pool <- pool %>% left_join(select(pool_allelic, c(snp, logFC, lrt_pval)), by = "snp") %>%
    rename(pval_allelic = lrt_pval)

In [21]:
head(pool)

snp,alpha_ref,alpha_alt,log2a_ref,log2a_alt,zscore_ref,zscore_alt,DNA_CPM_QC,is_enhancer,logFC,pval_allelic
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<dbl>,<dbl>
rs11583631,1.425401,1.328563,0.5113677,0.4098670,-0.098448693,-0.30463779,FALSE,NA,-0.04727376,6.748780e-01
rs11584419,1.476491,1.412684,0.5621723,0.4984384,0.004756118,-0.12471339,FALSE,NA,0.04456964,6.565204e-01
rs749435,2.005698,2.140291,1.0041041,1.0978068,0.902498810,1.09284713,TRUE,TRUE,-0.06529871,8.405816e-02
rs3789524,1.422514,1.491660,0.5084428,0.5769184,-0.104390459,0.03471129,TRUE,TRUE,0.04192126,1.242780e-03
rs3789522,1.602634,1.360524,0.6804446,0.4441627,0.245015024,-0.23496934,TRUE,TRUE,-0.16202368,2.199461e-19
rs11121478,1.132672,1.146398,0.1797297,0.1971084,-0.772140080,-0.73683690,TRUE,FALSE,-0.17450039,3.369730e-02


In [22]:
# calculate FDR for only enhancers
pool_enc <- pool %>% filter(is_enhancer == TRUE) %>% 
    mutate(fdr_allelic=p.adjust(pval_allelic, method = "BH")) %>% select(snp, fdr_allelic)

In [23]:
# add FDR values to table
pool <- pool %>% left_join(pool_enc, by = 'snp')

In [24]:
head(pool)

snp,alpha_ref,alpha_alt,log2a_ref,log2a_alt,zscore_ref,zscore_alt,DNA_CPM_QC,is_enhancer,logFC,pval_allelic,fdr_allelic
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>
rs11583631,1.425401,1.328563,0.5113677,0.4098670,-0.098448693,-0.30463779,FALSE,NA,-0.04727376,6.748780e-01,NA
rs11584419,1.476491,1.412684,0.5621723,0.4984384,0.004756118,-0.12471339,FALSE,NA,0.04456964,6.565204e-01,NA
rs749435,2.005698,2.140291,1.0041041,1.0978068,0.902498810,1.09284713,TRUE,TRUE,-0.06529871,8.405816e-02,1.350025e-01
rs3789524,1.422514,1.491660,0.5084428,0.5769184,-0.104390459,0.03471129,TRUE,TRUE,0.04192126,1.242780e-03,3.609169e-03
rs3789522,1.602634,1.360524,0.6804446,0.4441627,0.245015024,-0.23496934,TRUE,TRUE,-0.16202368,2.199461e-19,3.108571e-18
rs11121478,1.132672,1.146398,0.1797297,0.1971084,-0.772140080,-0.73683690,TRUE,FALSE,-0.17450039,3.369730e-02,NA


In [25]:
# find number of elements that are significant with FDR <= 0.05
sum(pool$fdr_allelic<=0.01, na.rm=TRUE)

[1] 85

In [26]:
# find allelic significance
pool_all <- pool %>% filter(is_enhancer) %>% mutate(is_allelic = (fdr_allelic <= 0.01) & (abs(logFC)>=log(1.1)))
pool1 <- pool %>% left_join(select(pool_all, c(snp, is_allelic)), by = "snp")

In [27]:
# save enhancer and allelic significance results
write.table(pool1, "../results/pool1_final_result.txt", 
            sep="\t", quote = FALSE, row.names = FALSE, col.names = TRUE)

## Pool 2

In [28]:
# load MPRAObjects
pool_alpha <- read.table("../results/pool2_mpranalyze_results.txt")

In [29]:
# load DNA CPM counts
pool_cpm <- read.table("../results/pool2_dna_counts_cpm.txt")

In [30]:
# load results from allelic significance
pool_allelic <- read.table("../results/pool2_mpranalyze_allelic_results_merged.txt", header=TRUE)

In [31]:
head(pool_alpha)

,snp,type,alpha
,<chr>,<chr>,<dbl>
1,rs67190025,ref,1.0790228
2,rs67190025,alt,1.1311419
3,rs7580924,ref,1.0332542
4,rs7580924,alt,0.8341374
5,rs13028959,ref,0.9788775
6,rs13028959,alt,0.9754385


In [32]:
head(pool_cpm)

,chrom,hg19_pos,snp,allele,mutation,testcre,V1_DNA_ref_count,V1_DNA_alt_count,V2_DNA_ref_count,V2_DNA_alt_count,V1_DNA_ref_CPM,V1_DNA_alt_CPM,V2_DNA_ref_CPM,V2_DNA_alt_CPM,DNA_CPM_QC
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
1,chr2,201172086,rs67190025,C,T,chr2_201172085_rs67190025_C_T,21,36,307,295,2.920649,5.006827,9.546576,9.17342,TRUE
2,chr2,201172628,rs7580924,G,C,chr2_201172627_rs7580924_G_C,245,263,868,1119,34.074242,36.577655,26.991623,34.79680,TRUE
3,chr2,201185919,rs13028959,C,A,chr2_201185918_rs13028959_C_A,5321,4688,19770,16641,740.036900,652.000186,614.774649,517.47420,TRUE
4,chr2,201194504,rs112467613,C,CAA,chr2_201194503_rs112467613_C_CAA,21922,17420,79508,68046,3048.879708,2422.748131,2472.407829,2115.98158,TRUE
5,chr3,38417345,rs196378,G,C,chr3_38417344_rs196378_G_C,155,138,1087,667,21.557173,19.192838,33.801722,20.74126,TRUE
6,chr3,38417367,rs169046,T,C,chr3_38417366_rs169046_T_C,277,188,1586,1383,38.524755,26.146765,49.318796,43.00624,TRUE


In [33]:
head(pool_allelic)

,snp,logFC,lrt_pval,lrt_nlog10_pval
,<chr>,<dbl>,<dbl>,<dbl>
1,rs67190025,0.0433978350,6.931219e-01,0.15919041
2,rs7580924,-0.0920718548,1.678202e-01,0.77515582
3,rs13028959,0.0006139366,8.018869e-01,0.09588687
4,rs112467613,0.1170404404,2.861114e-29,28.54346476
5,rs196378,-0.0951442512,1.688483e-01,0.77250318
6,rs169046,0.3165248123,2.197575e-06,5.65805630


In [34]:
head(pool_alpha)

,snp,type,alpha
,<chr>,<chr>,<dbl>
1,rs67190025,ref,1.0790228
2,rs67190025,alt,1.1311419
3,rs7580924,ref,1.0332542
4,rs7580924,alt,0.8341374
5,rs13028959,ref,0.9788775
6,rs13028959,alt,0.9754385


In [35]:
# log-transform alpha values
pool_alpha <- pool_alpha %>% mutate(log2a = log2(alpha))

In [36]:
# calculate z-score with alpha
pool_alpha <- pool_alpha %>% mutate(zscore = (log2a - mean(log2a)) / sd(log2a))

In [37]:
head(pool_alpha)

,snp,type,alpha,log2a,zscore
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,rs67190025,ref,1.0790228,0.10972531,0.14461201
2,rs67190025,alt,1.1311419,0.17777993,0.32379097
3,rs7580924,ref,1.0332542,0.04719521,-0.02002163
4,rs7580924,alt,0.8341374,-0.26164312,-0.83315286
5,rs13028959,ref,0.9788775,-0.03079980,-0.22537238
6,rs13028959,alt,0.9754385,-0.03587719,-0.23874048


In [38]:
# widen table to separate alleletypes and find enhancer significance
pool_alpha_w <- pool_alpha %>% 
    pivot_wider(names_from=type, values_from=c(alpha, log2a, zscore))

In [39]:
# select relevant columns from DNA CPM counts
pool_cpm <- pool_cpm %>% select(snp, DNA_CPM_QC)

In [40]:
head(pool_cpm)

,snp,DNA_CPM_QC
,<chr>,<lgl>
1,rs67190025,TRUE
2,rs7580924,TRUE
3,rs13028959,TRUE
4,rs112467613,TRUE
5,rs196378,TRUE
6,rs169046,TRUE


In [41]:
# add DNA CPM classifications to v1 pool1 table
pool_alpha_w <- pool_alpha_w %>% left_join(pool_cpm, by = c("snp"))

In [42]:
head(pool_alpha_w)

snp,alpha_ref,alpha_alt,log2a_ref,log2a_alt,zscore_ref,zscore_alt,DNA_CPM_QC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
rs67190025,1.0790228,1.1311419,0.10972531,0.17777993,0.14461201,0.32379097,TRUE
rs7580924,1.0332542,0.8341374,0.04719521,-0.26164312,-0.02002163,-0.83315286,TRUE
rs13028959,0.9788775,0.9754385,-0.03079980,-0.03587719,-0.22537238,-0.23874048,TRUE
rs112467613,0.9637650,1.0645680,-0.05324668,0.09026808,-0.28447210,0.09338364,TRUE
rs196378,1.3289014,1.2821125,0.41023406,0.35852281,0.93581252,0.79966352,TRUE
rs169046,1.3480345,1.6999456,0.43085739,0.76548860,0.99011108,1.87115163,TRUE


In [43]:
# classify variants that are enhancers
pool_enhancer <- pool_alpha_w %>% filter(DNA_CPM_QC) %>% mutate(is_enhancer=(zscore_ref > 0 | zscore_alt > 0))

In [44]:
# join enhancer classifications with overall table
pool <- pool_alpha_w %>% left_join(pool_enhancer)

Joining with `by = join_by(snp, alpha_ref, alpha_alt, log2a_ref, log2a_alt,
zscore_ref, zscore_alt, DNA_CPM_QC)`


In [45]:
head(pool)

snp,alpha_ref,alpha_alt,log2a_ref,log2a_alt,zscore_ref,zscore_alt,DNA_CPM_QC,is_enhancer
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>
rs67190025,1.0790228,1.1311419,0.10972531,0.17777993,0.14461201,0.32379097,TRUE,TRUE
rs7580924,1.0332542,0.8341374,0.04719521,-0.26164312,-0.02002163,-0.83315286,TRUE,FALSE
rs13028959,0.9788775,0.9754385,-0.03079980,-0.03587719,-0.22537238,-0.23874048,TRUE,FALSE
rs112467613,0.9637650,1.0645680,-0.05324668,0.09026808,-0.28447210,0.09338364,TRUE,TRUE
rs196378,1.3289014,1.2821125,0.41023406,0.35852281,0.93581252,0.79966352,TRUE,TRUE
rs169046,1.3480345,1.6999456,0.43085739,0.76548860,0.99011108,1.87115163,TRUE,TRUE


In [46]:
# extract allelic p-values
pool <- pool %>% left_join(select(pool_allelic, c(snp, logFC, lrt_pval)), by = "snp") %>%
    rename(pval_allelic = lrt_pval)

In [47]:
head(pool)

snp,alpha_ref,alpha_alt,log2a_ref,log2a_alt,zscore_ref,zscore_alt,DNA_CPM_QC,is_enhancer,logFC,pval_allelic
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<dbl>,<dbl>
rs67190025,1.0790228,1.1311419,0.10972531,0.17777993,0.14461201,0.32379097,TRUE,TRUE,0.0433978350,6.931219e-01
rs7580924,1.0332542,0.8341374,0.04719521,-0.26164312,-0.02002163,-0.83315286,TRUE,FALSE,-0.0920718548,1.678202e-01
rs13028959,0.9788775,0.9754385,-0.03079980,-0.03587719,-0.22537238,-0.23874048,TRUE,FALSE,0.0006139366,8.018869e-01
rs112467613,0.9637650,1.0645680,-0.05324668,0.09026808,-0.28447210,0.09338364,TRUE,TRUE,0.1170404404,2.861114e-29
rs196378,1.3289014,1.2821125,0.41023406,0.35852281,0.93581252,0.79966352,TRUE,TRUE,-0.0951442512,1.688483e-01
rs169046,1.3480345,1.6999456,0.43085739,0.76548860,0.99011108,1.87115163,TRUE,TRUE,0.3165248123,2.197575e-06


In [48]:
# calculate FDR for only enhancers
pool_enc <- pool %>% filter(is_enhancer == TRUE) %>% 
    mutate(fdr_allelic=p.adjust(pval_allelic, method = "BH")) %>% select(snp, fdr_allelic)

In [49]:
# add FDR values to table
pool <- pool %>% left_join(pool_enc, by = 'snp')

In [50]:
head(pool)

snp,alpha_ref,alpha_alt,log2a_ref,log2a_alt,zscore_ref,zscore_alt,DNA_CPM_QC,is_enhancer,logFC,pval_allelic,fdr_allelic
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>
rs67190025,1.0790228,1.1311419,0.10972531,0.17777993,0.14461201,0.32379097,TRUE,TRUE,0.0433978350,6.931219e-01,7.994920e-01
rs7580924,1.0332542,0.8341374,0.04719521,-0.26164312,-0.02002163,-0.83315286,TRUE,FALSE,-0.0920718548,1.678202e-01,NA
rs13028959,0.9788775,0.9754385,-0.03079980,-0.03587719,-0.22537238,-0.23874048,TRUE,FALSE,0.0006139366,8.018869e-01,NA
rs112467613,0.9637650,1.0645680,-0.05324668,0.09026808,-0.28447210,0.09338364,TRUE,TRUE,0.1170404404,2.861114e-29,9.523424e-28
rs196378,1.3289014,1.2821125,0.41023406,0.35852281,0.93581252,0.79966352,TRUE,TRUE,-0.0951442512,1.688483e-01,2.676304e-01
rs169046,1.3480345,1.6999456,0.43085739,0.76548860,0.99011108,1.87115163,TRUE,TRUE,0.3165248123,2.197575e-06,1.219131e-05


In [51]:
# find number of elements that are significant with FDR <= 0.05
sum(pool$fdr_allelic<=0.01, na.rm=TRUE)

[1] 83

In [52]:
# find allelic significance
pool_all <- pool %>% filter(is_enhancer) %>% mutate(is_allelic = (fdr_allelic <= 0.01) & (abs(logFC)>=log(1.1)))
pool2 <- pool %>% left_join(select(pool_all, c(snp, is_allelic)), by = "snp")

In [53]:
head(pool2)

snp,alpha_ref,alpha_alt,log2a_ref,log2a_alt,zscore_ref,zscore_alt,DNA_CPM_QC,is_enhancer,logFC,pval_allelic,fdr_allelic,is_allelic
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<lgl>
rs67190025,1.0790228,1.1311419,0.10972531,0.17777993,0.14461201,0.32379097,TRUE,TRUE,0.0433978350,6.931219e-01,7.994920e-01,FALSE
rs7580924,1.0332542,0.8341374,0.04719521,-0.26164312,-0.02002163,-0.83315286,TRUE,FALSE,-0.0920718548,1.678202e-01,NA,NA
rs13028959,0.9788775,0.9754385,-0.03079980,-0.03587719,-0.22537238,-0.23874048,TRUE,FALSE,0.0006139366,8.018869e-01,NA,NA
rs112467613,0.9637650,1.0645680,-0.05324668,0.09026808,-0.28447210,0.09338364,TRUE,TRUE,0.1170404404,2.861114e-29,9.523424e-28,TRUE
rs196378,1.3289014,1.2821125,0.41023406,0.35852281,0.93581252,0.79966352,TRUE,TRUE,-0.0951442512,1.688483e-01,2.676304e-01,FALSE
rs169046,1.3480345,1.6999456,0.43085739,0.76548860,0.99011108,1.87115163,TRUE,TRUE,0.3165248123,2.197575e-06,1.219131e-05,TRUE


In [54]:
# save enhancer and allelic significance results
write.table(pool2, "../results/pool2_final_result.txt", 
            sep="\t", quote = FALSE, row.names = FALSE, col.names = TRUE)